오늘의 구현순서

1) 데이터 준비 (배경과 사람거리가 있는 것으로다가~)
2)시멘틱 세그멘테이션으로 피사체와 배경을 분리 
3)블러링 기술로 배경을 흐리게 만들기 
4)피사체를 배경의 원래 위치로 합성 

In [2]:
#패키지 불러오기 

import cv2
import numpy as np
import os
from glob import glob
from os.path import join
import tarfile
import urllib # 웹에서 데이터를 다운받을 때 사용합니다. 

from matplotlib import pyplot as plt
import tensorflow as tf

In [11]:
# 이미지 저장하기 

#이미지 저장은 저장경로를 변수에 지정하고 그것이 이미지 파일이기에 cv2로 읽습니다. 
import os 
img_path = os.getenv('HOME') + '/aiffel/human_segmentation/images/market'
img_orig = cv2.imread(img_path)
print (img_orig.shape)
len(img_orig[0]) #가로 세로..음...가로 183개, 세로 275인 RGB파일 

(183, 275, 3)


275

# 세그멘테이션으로 사람 분리하기 
![image](https://user-images.githubusercontent.com/68840414/108790657-9b068b00-75c0-11eb-9a05-9b1dddbc115e.png)

segmentation은 input이미지의 
* instance segmentation : 같은 클래스 분류 
* semantic segmentaion: 같은 클래스 분류 하지 않음

원본이미지 

![image](https://user-images.githubusercontent.com/68840414/108791202-dd7c9780-75c1-11eb-9b8d-8d45205028db.png)

인스턴스와 시멘틱 segmentation
![image](https://user-images.githubusercontent.com/68840414/108791183-d5bcf300-75c1-11eb-9a3f-3043059a7d1b.png)





#### 질문: 인스턴스 세그멘테이션 데이터 셋에 라벨이 존재한다면 시맨틱 세그멘테이션을 학습시킬 수 있을까요?
인스턴스 id뿐만 아니라 클래스 라벨이 존재해야 시맨틱 세그멘테이션이 학습이 가능합니다. 그러나 대부분의 데이터셋은 인스턴스 라벨이 있는 경우 클래스 라벨도 함께 존재합니다. 

이미지 세그멘테이션의 간단한 알고리즘: 워터쉐드 세그멘테이션(watershed segmentation)

특징: 경계를 또렷하게 보이기 위해서 grmayscale로 변환하여 0~255의 값을 가집니다. 픽셀값을 사용해서 각 위치의 높낮이를 구분하는데 낮은 부분부터 서서히 
    '물'을 채워 나간다고 생각할 때 각 영역에서 점점 물이 차 오느라다 넘치는 지점이 생기고, 그 부분을 경계선으로 만들면 물체를 서로 구분 할 수 있게 됩니다. 
    
    

#segmentation FCN, SegNet, U-net등 많은 모델이 있지만 여기서는 DeepLab v3+를 사용합니다. 

FCN: 2014년 논문 pixel level classification  pooling으로 인해 해상도가 감소했는데 , skip connection을 사용하였다. 

DeepLab v3+ 성능표

![image](https://user-images.githubusercontent.com/68840414/109092565-1fd8dc80-775a-11eb-947b-d17ca7f2af7d.png)


atrous convolution을 사용하는 이유는?
depthwise separable convolution이 효율적인 이유는? 

In [ ]:
#deep lab 모델 클래스를 가져옵니다. 

class DeepLabModel(object):
    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    INPUT_SIZE = 513
    FROZEN_GRAPH_NAME = 'frozen_inference_graph'

    # __init__()에서 모델 구조를 직접 구현하는 대신, tar file에서 읽어들인 그래프구조 graph_def를 
    # tf.compat.v1.import_graph_def를 통해 불러들여 활용하게 됩니다. 
    def __init__(self, tarball_path):
        self.graph = tf.Graph()
        graph_def = None
        tar_file = tarfile.open(tarball_path)
        for tar_info in tar_file.getmembers():
            if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
                file_handle = tar_file.extractfile(tar_info)
                graph_def = tf.compat.v1.GraphDef.FromString(file_handle.read())
                break
        tar_file.close()

        with self.graph.as_default():
    	    tf.compat.v1.import_graph_def(graph_def, name='')

        self.sess = tf.compat.v1.Session(graph=self.graph)

    # 이미지를 전처리하여 Tensorflow 입력으로 사용 가능한 shape의 Numpy Array로 변환합니다.
    def preprocess(self, img_orig):
        height, width = img_orig.shape[:2]
        resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
        target_size = (int(resize_ratio * width), int(resize_ratio * height))
        resized_image = cv2.resize(img_orig, target_size)
        resized_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        img_input = resized_rgb
        return img_input
        
    def run(self, image):
        img_input = self.preprocess(image)

        # Tensorflow V1에서는 model(input) 방식이 아니라 sess.run(feed_dict={input...}) 방식을 활용합니다.
        batch_seg_map = self.sess.run(
            self.OUTPUT_TENSOR_NAME,
            feed_dict={self.INPUT_TENSOR_NAME: [img_input]})

        seg_map = batch_seg_map[0]
        return cv2.cvtColor(img_input, cv2.COLOR_RGB2BGR), seg_map